In [30]:
import numpy as np

In [49]:
def CalculateDiffusionIncrements(CorrMatrix, dt, N):
    '''
    Given the sqare root of the correlation matrix and the size of time step, 
    calculate the values of dW_t or dB_t for all diffusion brownian motions in our model. 
    
    PARAMS:
    
    CorrMatrix: value of stochastic correlation matrix observation at time t.
    Size of the matrix is 2N+1 x 2N + 1. 
    
    dt: number, the size of time step of the simulation. 
    
    N:  the number of particles in our system. 
    '''
    iidStandartNormalVector = np.random.randn(2*N + 1)
    Increment = np.linalg.cholesky(CorrMatrix) @ iidStandartNormalVector * np.sqrt(dt)
    
    return Increment

In [59]:
class coef_function1d(object):
    '''
    1-dimensional coeffitient function. It depends on only 1 argument.
    Contains its value, first and second derivatives in point x. 
    '''
    
    def __init__(self, value, FirstDerivative, SecondDerivative):
        self.value = value
        self.dx    = FirstDerivative
        self.dxdx  = SecondDerivative
    
    def ReturnValue(self, x):
        return self.value(x)
    
    def dx(self, x):
        return self.FirstDerivative(x)
    
    def dxdx(self, x):
        return self.SecondDerivative(x)
    
    
class coef_function2d(object):
    '''
    2-dimensional coefficient function. Usually takes
    time-moment and process value as arguments. Both are real numbers.
    '''
    
    def __init__(self, value, Dt, Dx, DxDx):
        self.value = value
        self.Dt    = Dt
        self.Dx    = Dx
        self.DxDx  = Dxdx
    
    def value(self, t, x):
        return value(t, x)

    def dt(self, t, x):
        return self.Dt(t, x)
    
    def dx(self, t, x):
        return self.Dx(t, x)
    
    def dxdx(self, t, x):
        return self.DxDx(t, x)
    
class coef_function3d(object):
    '''
    3-dimensional coefficient function. Logic remains the same. 
    '''
    
    def __init__(self, value, Dt, Dx, Dy, DxDy, DxDx, DyDy):
        self.value = value
        self.Dt    = Dt
        self.Dx    = Dx
        self.Dy    = Dy
        self.DxDy  = Dxdy
        self.DxDx  = Dxdx
        self.DyDy  = Dydy
        
    def value(self, t, x, y):
        return self.value(t, x, y)
    
    def dt(self, t, x, y):
        return self.Dt(t, x, y)
    
    def dx(self, t, x, y):
        return self.Dx(t, x, y)
    
    def dy(self, t, x, y):
        return self.Dy(t, x, y)
    
    def dxdy(self, t, x, y):
        return self.DxDy(t, x, y)
    
    def dxdx(self, t, x, y):
        return self.DxDx(t, x, y)
    
    def dydy(self, t, x, y):
        return self.DyDy(t, x, y)   
    
    
class mean_reversion(object):
    '''
    Class for correlation mean-reverted drift function.
    It depends on N+1 arguments: nu1, ... , nuN, rho. 
    '''
    def __init__(self, value, gradient, hessian):
        self.value    = value
        self.gradient = gradient
        self.hessian  = hessian
        
    def dx(self, NuVector, rho, coordinate):
        return self.gradient[coordinate](NuVector, rho)
    
    def dxdx(self, NuVector, rho, i, j):
        return self.gessian[i][j](NuVector, rho)

In [ ]:
class ParticleProcess(object):
    '''
    This process describes the behaviour of the asset price. 
    
    The process can be described by an SDE:
    dS = drift_coef(t, S)dt + diffusion_coef(t, S, Nu)dWt, where Nu is the volatility process.
    
    NOT IMPLEMENTED YET
    
    PARAMS:
    S0:             the initial value of our diffusion. A number or a vector of numbers.
    drift_coef:     2d function, depends on time and process value. 
    diffusion_coef: 3d function, depends on time, process value and volatility value. 
    '''
    
    def __init__(self, S0, drift_coef, diffusion_coef):
        self.S0             = S0
        self.drift_coef     = drift_coef
        self.diffusion_coef = diffusion_coef
        

    

class VolatilityProcess(object):
    '''
    This process describes the behaviour of the asset price. 
    
    The process can be described by an SDE:
    dNu = drift_coef(t, Nu)dt + diffusion_coef(t, Nu)dBt, where Nu is the volatility process.
    
    NOT IMPLEMENTED YET
    
    PARAMS:
    Nu0:            the initial value of our diffusion. A number or a vector of numbers.
    drift_coef:     2d function, depends on time and volatility value. 
    diffusion_coef: 2d function, depends on time, and volatility value. 
    '''
    
    def __init__(self, Nu0, drift_coef, diffusion_coef):
        self.Nu0            = Nu0
        self.drift_coef     = drift_coef
        self.diffusion_coef = diffusion_coef

        
    

class CorrelationProcess(object):
    '''
    This process describes the behaviour of the asset price. 
    
    The process can be described by an SDE:
    dNu = drift_coef(t, Nu)dt + diffusion_coef(t, Nu)dBt, where Nu is the volatility process.
    
    NOT IMPLEMENTED YET
    
    PARAMS:
    rho0:           the initial value of our process. A number or a vector of numbers.
    mean_reversion: Psy(Nu1, ... , NuN) - rho_t. 
    diffusion_coef: 1d function, depends only on correlation value. 
    '''
    def __init__(self, rho0, alpha, mean_reversion, diffustion_coef):
        self.rho0           = rho0
        self.alpha          = alpha
        self.drift_coef     = self.alpha * mean_reversion
        self.diffusion_coef = diffusion_coef

In [ ]:
def DoubleIntegral(BrownianIncrements, V, dt, i, j):
    '''
    Calculates the values of double integrals that appear in second-order refinement.
    
    PARAMS:
    BrownianIncrements: vector of all Brownian increments. 
    V: matrix of helping variables. V[i][j] = -V[j][i] for all i < j
       V[i][i] = h for all i. 
       V[i][j] = h with prob 0.5 and -h otherwise for all i < j.
    dt: size of time step
    i, j: number of corresponding Brownian motions with which the integral appeared. 
    '''
    if   (i == 0 and j == 0):
        return 0.5 * dt ** 2
    elif (i == 0):
        0.5 * dt * BrownianIncrements[j]
    elif (j == 0):
        0.5 * dt * BrownianIncrements[i]
    elif (i > 0 and j > 0):
        return 0.5 * (BrownianIncrements[i] * BrownianIncrements[j] - V[i][j])
    else: 
        print("Error in i, j.\n")

In [ ]:
def ParticleDriftItoOperator(ParticleProcess, t, S, Nu):
    
    '''
    Function that returns values of two operators that are used in Ito-Doeblin formula. 
    The value is calculated in the point (t, S, Nu). 
    
    So, particle drift mu(t, S) can be approximated as:
    dmu(t, D) = DriftValue(t, S, Nu)*dt + DiffusionValue*dWt,
    where dt, dWt are taken from dSt. 
    
    PARAMS: 
    ParticleProcess: class of an SDE that describes our particle. 
    t: time moment
    S: particle value at time t.
    Nu: volatility value at time t. 
    
    RETURNS: 
    Two numbers, the values of the corresponding operators. 
    '''  
    DriftValue = ParticleProcess.drift_coef.dt(t, S) + 
                 ParticleProcess.drift_coef.value(t, S) * ParticleProcess.drift_coef.dx(t, S) +
                 0.5 * (ParticleProcess.diffusion_coef(t, S, Nu) ** 2) * ParticleProcess.drift_coef.dxdx(t,S)
            
    DiffusionValue = ParticleProcess.diffusion_coef(t, S, Nu) * ParticleProcess.drift_coef.dx(t, S)
    
    return DriftValue, DiffusionValue


def ParticleDiffusionItoOperator(ParticleProcess, VolatilityProcess, Corr, t, S, Nu):
    '''
    PARAMS:
    ParticleProcess: an SDE class that describes our asset price. 
    VolatilityProcess: an SDE class that describes the volatility of the asset price. 
    t: current time moment
    Corr: correlation between dWt and dBt: dWtdBt = Corr * dt at time point t. 
    S: asset price at time t
    Nu: volatility value at time t. 
    
    RETURNS:
    DriftValue -- the value of the drift operator at time t. 
    DiffusionSigmaValue, DiffusionDvalue -- diffusion operators values at time t. 
    '''
    DriftValue = ParticleProcess.diffusion_coef.dt(t, S, Nu) + 
                 ParticleProcess.drift_coef.value(t, S) * ParticleProcess.diffusion_coef.dx(t, S, Nu) + 
                 VolatilityProcess.drift_coef.value(t, Nu) * ParticleProcess.diffusion_coef.dy(t, S, Nu) +
                 0.5 * (ParticleProcess.diffusion_coef.value(t, S, Nu)**2) 
                 * ParticleProcess.diffusion_coef.dxdx(t, S, Nu) + 
                0.5 * (VolatilityProcess.diffusion_coef.value(t, Nu)**2) 
                     * ParticleProcess.diffusion_coef.dydy(t, S, Nu) + 
                Corr * ParticleProcess.diffusion_coef.value(t, S, Nu) * 
                VolatilityProcess.diffusion_coef.value(t, Nu) * ParticleProcess.diffusion_coef.dxdy(t, S, Nu)
    
    DiffusionSigmaValue = ParticleProcess.diffusion_coef.value(t, S, Nu) 
                        * ParticleProcess.diffusion_coef.dx(t, S, Nu) 
        
    DiffusionDValue     = VolatilityProcess.diffusion_coef.value(t, Nu)  
                        * ParticleProcess.diffusion_coef.dy(t, S, Nu)
    
    return DriftValue, DiffusionSigmaValue, DiffusionDValue
    

def VolatilityDriftItoOperator(VolatilityProcess, t, Nu):
    '''
    Here, logic remains the same with the previous operators. This one
    has a less complicated structure.  
    '''
    DriftValue = VolatilityProcess.drift_coef.dt(t, Nu) +  
                 VolatilityProcess.drift_coef.value(t, Nu) * VolatilityProcess.drift_coef.dx(t, Nu)  +
                 0.5 * (VolatilityProcess.diffusion_coef.value(t, Nu) ** 2) 
                 * VolatilityProcess.drift_coef.dxdx(t, Nu)
    
    DiffusionValue = VolatilityProcess.diffusion_coef.value(t, Nu) * VolatilityProcess.drift_coef.dx(t, Nu)
    
    return DriftValue, DiffusionValue
    
    
def VolatilityDiffusionItoOperator(VolatilityProcess, t, Nu):
    '''
    Here, logic remains the same with the previous operators. This one
    has a less complicated structure.  
    '''
    DriftValue = VolatilityProcess.diffusion_coef.dt(t, Nu) +  
                 VolatilityProcess.drift_coef.value(t, Nu) * VolatilityProcess.diffusion_coef.dx(t, Nu)  +
                 0.5 * (VolatilityProcess.diffusion_coef.value(t, Nu) ** 2) 
                 * VolatilityProcess.diffusion_coef.dxdx(t, Nu)
    
    DiffusionValue = VolatilityProcess.diffusion_coef.value(t, Nu) * VolatilityProcess.diffusion_coef.dx(t, Nu)
    
    return DriftValue, DiffusionValue
                
def CorrelationDriftItoOperator(CorrelationProcess, VolatilityVectorProcess, t, NuVector, rho, N):
    '''
    NEED TO CHECK
    '''
    DriftValue = (-1) * CorrelationProcess.drift_coef.value(NuVector, rho)
    for k in range(N):
        DriftValue += VolatilityVectorProcess[k].drift_coef.value(t, NuVector[k]) *
                      CorrelationProcess.drift_coef.dx(NuVector, rho, k)       
    for k in range(N):
        for l in range(N):
            DriftValue += 0.5 * VolatilityVectorProcess[k].diffusion_coef.value(t, NuVector[k]) 
                       * VolatilityVectorProcess[l].diffusion_coef.value(t, NuVector[l]) 
                       * CorrelationProcess.drift_coef.dxdx(NuVector, rho, k, l)
                    
    DiffusionDVectorValue = []
    for k in range(N):
        DiffusionDVectorValue.append(VolatilityVectorProcess[k].diffusion_coef.value(t, NuVector[k]) 
                                     * CorrelationProcess.drift_coef.dx(NuVector, rho, k))
    
    DiffusionOmegaVectorValue = - CorrelationProcess.diffusion_coef.value(rho)*CorrelationProcess.alpha
    
    return DriftValue, DiffusionDVectorValue, DiffusionOmegaVectorValue
                        
    
def CorrelationDiffusionItoOperator(CorrelationProcess, rho, Nus):
    '''
    NEED TO CHECK
    '''    
    DriftValue = CorrelationProcess.drift_coef.value(rho) * CorrelationProcess.diffusion_coef.dx(rho) +
                 0.5 * (CorrelationProcess.diffusion_coef.value(rho) ** 2) 
                 * CorrelationProcess.diffusion_coef.dxdx(rho)
            
    DiffusionValue = CorrelationProcess.diffusion_coef.value(rho) * CorrelationProcess.diffusion_coef.dx(rho)
    
    return DriftValue, DiffusionValue

In [37]:
def ConstructParticleCorrelationMatrix(rho, N):
    '''
    CAN BE OPTIMIZED!!!
    
    
    PARAMS:
    rho -- correlation process value at time t. 
    N   -- number of particles in our model.
    
    RETURNS:
    The simple particle-particle correlation matrix with correlation value rho. 
    Is further used in constructing the general correlation matrix. 
    '''
    rho_vector = rho * np.ones(N).reshape(N, 1)
    ones_vector = np.ones(N).reshape(N, 1)
    
    return (rho_vector @ ones_vector.T) - np.diag(np.ones(N) * rho) + np.diag(np.ones(N))
    

def CalculateGeneralCorrMatrix(rho, rho_skew, ParticleVolCorrMatrix, VolCorrMatrix, N):
    '''
    Calculates the correlation matrix of the whole model. This matrix describes all of the 
    crosscorrelations between brownian motions and is further used for the calculation of
    the brownian increments of the model. 
    
    PARAMS:
    rho                   -- correlation process value at time t. 
    rho_skew              -- correlation between any brownian motion with the motion of the correlation process. 
    ParticleVolCorrMatrix -- dW^idB^j/dt are the elems of this one. 
    VolCorrMatrix         -- dB^idB^j are the elems of that one. 
    N                     -- number of particles in the model. 
    
    RETURNS:
    2N+1 x 2N+1 correlation matrix. 
    '''
    
    ParticleCorrMatrix = ConstructParticleCorrelationMatrix(rho, N)
    L1 = np.concatenate((ParticleCorrMatrix, ParticleVolCorrMatrix), axis=1)
    L2 = np.concatenate((ParticleVolCorrMatrix, VolCorrMatrix), axis=1)
    
    MainBlock = np.concatenate((L1, L2), axis=0)
    skew_vector = np.ones(2*N).reshape(2*N, 1) * rho_skew
    
    ExtendedMainBlock = np.concatenate((MainBlock, skew_vector), axis=1)
    extended_skew_vector = np.concatenate((skew_vector, np.ones(1).reshape(1, 1)), axis=0)
    
    return np.concatenate((ExtendedMainBlock, extended_skew_vector.T), axis=0)

In [57]:
def Jackson(x):
    return x**2 + x + 1

def dxJackson(x):
    return 2*x + 1

def dxdxJackson(x):
    return 2

f = function1d(Jackson, dxJackson, dxdxJackson)

## Simple Euler Scheme. 

In [28]:
def dNu(dt, prev_t, prev_Nu, Nu_SDE, dB):
    '''
    1-d function.
    
    PARAMS:
    dt      -- size of the time step.
    prev_t  -- time point at which we freeze the coefficient.
    prev_Nu -- Nu_t
    Nu-SDE  -- the class of the volatility process that we are approximating. 
    dB      -- the corresponding Brownian increment. 
    
    RETURNS:
    The approximated value of volatility increment on time: [t, t + dt]. 
    '''
    return Nu_SDE.drift_coef.value(prev_t, prev_Nu) * dt + Nu_SDE.diffusion_coef.value(prev_t, prev_Nu) * dB
    
def drho(dt, prev_Nu_vector, prev_rho, rho_SDE, dWhat):
    '''
    Logic is the same.
    '''
    return rho_SDE.drift_coef.value(prev_Nu_vector, prev_rho) * dt + rho_SDE.diffusion_coef.value(prev_rho) * dWhat
    
def dS(dt, prev_t, prev_S, prev_Nu, S_SDE, dW):
    '''
    Logic remains the same. 
    '''
    return S_SDE.drift_coef.value(prev_t, prev_S) * dt
    + S_SDE.diffusion_coef.value(prev_t, prev_S, prev_Nu) * dW

In [22]:
class StochasticCorrModel(object):
    
    def __init__(Particles, Vols, Corr_Process, VolCorrMatrix, ParticleVolCorrMatrix, rho_skew):
        '''
        Attributes:
        Particles    -- the N-dimensional vector-process that describes asset prices. 
        Vols         -- the N-dimensional vector-process that descrives volatilities.
        Corr_Process -- the 1-d process that describes the correlation between brownian motions. 
        
        GeneralCorrMatrix  -- matrix of size 2N+1 x 2N+1 that contains all the correlations. 
        '''
        self.particles         = Particles
        self.vols              = vols
        self.corr_process      = Corr_Process
       
        self.N                 = len(Particles)
        
        self.VolCorrMatrix             = VolCorrMatrix
        self.ParticleVolCorrMatrix     = ParticleVolCorrMatrix
        
    def GeneralCorrMatrix(self, cur_rho):
        '''
        This method returns the current value of the General stochastic correlation matrix. 
        
        RETURNS: np.array of size 2N+1x2N+1. 
        '''
        return CalculateGeneralCorrMatrix(cur_rho, self.rho_skew, 
                                          self.ParticleVolCorrMatrix, self.VolCorrMatrix, self.N)

In [41]:
def SimpleEulerMemoryInitialization(Memory, S0, Nu0, rho0):
    Memory.T[0] = np.concatenate((S0, Nu0, np.array([rho0])), axis=0)
    

def SimpleEulerSimulation(model, dt, n, S0, Nu0, rho0):
    '''
    PARAMS:
    model -- StochasticCorrModel that contains all processes and the correlation matrix. 
    dt    -- time step size
    n     -- number  of iterations. ndt = T if we are simulating on the interval [0, T]. 
    '''
    # Array that contains all trajectories. Initialization. 
    Memory = np.zeros((2*model.N + 1) * (n + 1)).reshape(2*model.N + 1, n + 1) 
    SimpleEulerMemoryInitialization(Memory, S0, Nu0, rho0)
    
    for iteration in range(n):
        Gaussian_increments = CalculateDiffusionIncrements(model.GeneralCorrMatrix(Memory[-1][iteration]),
                                                           dt, model.N) 
        
        for i in range(model.N):        # Generate volatilities.
            Memory[i][iteration + 1] = Memory[i][iteration] + dS(dt, iteration*dt, Memory[i][iteration], 
                                                              Memory[model.N + i][iteration], 
                                                              model.particles[i], Gaussian_increments[i])
            
        for i in range(N, model.N * 2): # Generate particles. 
            Memory[i][iteration + 1] = Memory[i][iteration] + dNu(dt, iteration*dt, Memory[i][iteration], 
                                                                  model.vols[i], Gaussian_increments[i])
            
        Memory[-1][iteration + 1] = Memory[-1][iteration] + drho(dt, 
                                                                 Memory[model.N:model.N*2].T[iteration],
                                                                 Memory[-1][iteration],
                                                                 model.corr_process, 
                                                                 Gaussian_increments[-1])  
        
    return Memory

In [43]:
Memory = np.zeros(5*6).reshape(5, 6)
S0 = np.array([1, 1])
Nu0 = 2*np.array([1, 1])
rho0 = np.array([69])

SimpleEulerMemoryInitialization(Memory, S0, Nu0, rho0)

Memory

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  0.,  0.,  0.,  0.],
       [69.,  0.,  0.,  0.,  0.,  0.]])